In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

phases = dr.CLIPS

In [ ]:
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]


label_type = dr.SelfReports.AROUSAL
self_report_means = dr.get_mean_self_reports(label_type)
x, y = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=label_type)
print(x)


In [45]:
importlib.reload(train)
importlib.reload(dr)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]


models = {
    "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

label_type = dr.SelfReports.AROUSAL
threshold = "fixed"

x, y = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=label_type, threshold=threshold, normalize=True)
x = x.drop(["phaseId"], axis=1)
inds = pd.isnull(x).any(axis=1).to_numpy().nonzero()[0]
x = x.drop(labels=inds, axis=0)
y = y.drop(labels=inds, axis=0)

acc_results = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
reports = {
    "SVM": [],
    "LogReg": [],
    "XGB": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=True, save_metrics=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])

for model_name in acc_results.keys():
    acc = np.mean(acc_results[model_name])
    print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
    if acc > 0.6:
        print(f"Model evaluation metrics for {model_name}:")
        p = np.mean([report["precision"] for report in reports[model_name]])
        r = np.mean([report["recall"] for report in reports[model_name]])
        f1 = np.mean([report["f1"] for report in reports[model_name]])
        auc = np.mean([report["auc"] for report in reports[model_name]])
        report = reports[model_name]
        print(f"\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}")
print("\n")

test subjects: [57, 13, 10, 5, 25, 9, 43, 24]
y_train:
1    1119
0     195
Name: label, dtype: int64
y_test:
1    200
0     33
Name: label, dtype: int64
Model SVM, Predictions: [1], [233]
Model LogReg, Predictions: [0 1], [  1 232]
Model XGB, Predictions: [0 1], [  4 229]
test subjects: [49, 19, 41, 51, 30, 8, 57, 48]
y_train:
1    1134
0     196
Name: label, dtype: int64
y_test:
1    185
0     32
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [  3 214]
Model LogReg, Predictions: [1], [217]
Model XGB, Predictions: [0 1], [  7 210]
test subjects: [50, 13, 38, 51, 20, 42, 57, 55]
y_train:
1    1126
0     206
Name: label, dtype: int64
y_test:
1    193
0     22
Name: label, dtype: int64
Model SVM, Predictions: [1], [215]
Model LogReg, Predictions: [1], [215]
Model XGB, Predictions: [1], [215]
test subjects: [45, 31, 34, 30, 16, 47, 56, 1]
y_train:
1    1135
0     203
Name: label, dtype: int64
y_test:
1    184
0     25
Name: label, dtype: int64
Model SVM, Predictions: [0 1], [ 31 